# GRIDSEARCH

In [7]:
dicts = {'test':np.array([1,2,3])}
dicts



{'test': array([1, 2, 3])}

In [3]:

import sys
sys.path.append("../../src")
import features.feature_utils as fu
import models.model_utils as mu
import plots.plot_utils as pu
import log_utils as log
import math
from sklearn.model_selection import GridSearchCV
import numpy as np


def runGS(data, modelCore, modelOpt, parLabel, parRange, seqRegions, k=5, kInner= 4, pw=False, shuffle=True):

    dfDataset, featureBox = fu.CreateFeaturesFromData(data, seqRegions, pw, shuffle=shuffle)
    X = featureBox.values
    y = [math.sqrt(math.sqrt(u)) for u in dfDataset['mean_score']]

    parModel = mu.SetupModel(modelCore, modelOpt)
    parSet = mu.GetParameterSet(parLabel, parRange)
    model = mu.SelectModel(**parModel)
    print(featureBox.shape)
    print(parSet)
    GS = GridSearchCV(model, parSet, cv=k)
    GS.fit(X,y)

    fig1, ax = pu.ScatterPvT(GS, X, y)
    figures = [fig1]
    if len(parLabel) is 2:
        fig2, ax = pu.HeatmapCVDoublePar(GS)
        figures.append(fig2)
    if len(parLabel) is 1:
        fig2, ax = pu.HeatmapCVDoublePar(GS)
        figures.append(fig2)


    results = [str(GS.best_estimator_.get_params()),str(GS.best_score_),str(GS.cv_results_)]

    return results, figures

In [4]:

model = ['forestReg',None, None]
modelOpt= {}
parLabel = ['min_samples_split','max_depth']
parRange = [10,10] 
seqRegions = [[-7,12],[-6,11]]
k = 5 
kInner = 5
pw=False
data = '../../data/external/mut_rand_mod_lib.csv'

runGS(data, model, modelOpt, parLabel, parRange, seqRegions, k=k, kInner= kInner, pw=pw)

(254, 175)
{'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10]), 'min_samples_split': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11])}


(["{'n_jobs': 12, 'criterion': 'mse', 'min_samples_split': 4, 'n_estimators': 50, 'min_samples_leaf': 1, 'min_impurity_split': 1e-07, 'verbose': 0, 'max_depth': 9, 'random_state': None, 'min_weight_fraction_leaf': 0.0, 'oob_score': False, 'max_features': None, 'max_leaf_nodes': None, 'bootstrap': True, 'warm_start': False}",
  '0.663203504808',
  "{'std_train_score': array([ 0.02381572,  0.02344105,  0.02281454,  0.0250568 ,  0.01929344,\n        0.02092272,  0.02388746,  0.0223102 ,  0.01477994,  0.02154218,\n        0.03148329,  0.02489873,  0.0226664 ,  0.02987762,  0.01789338,\n        0.03576455,  0.02477844,  0.02128343,  0.02915898,  0.03617381,\n        0.01355309,  0.01912877,  0.01640208,  0.01604899,  0.01820813,\n        0.0178282 ,  0.0129774 ,  0.01714286,  0.01455894,  0.01686691,\n        0.01318939,  0.01050799,  0.01529344,  0.01365781,  0.01182996,\n        0.01263496,  0.01203441,  0.01227606,  0.00958874,  0.01212398,\n        0.01015977,  0.00916433,  0.01218227, 

# LEARNING CURVE IN SAMPLE

In [1]:
import sys
sys.path.append("../../src")
import features.feature_utils as fu
import models.model_utils as mu
import plots.plot_utils as pu
import pandas as pd
import math
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import logging
import json
import datetime as dt
import numpy as np

model = ['forestReg',None,None]
modelOpt = {}
k= 5
pw = False
step = 1
seqRegions = [[-7,12],[-6,11]]
data= '../../data/external/mut_rand_mod_lib.csv'

dfDatasetShuffled , featureBox = fu.CreateFeaturesFromData(data, seqRegions, pw, shuffle=True)
if pw is True:    
    y = dfDatasetShuffled['rank']
else:
    y = np.array([math.sqrt(math.sqrt(u)) for u in dfDatasetShuffled['mean_score']])
print(y)
parModel = mu.SetupModel(model, modelOpt)

scores = mu.LearningCurveInSample(dfDatasetShuffled, featureBox, y ,parModel, k=k, pw=pw, step=step)

fig, ax1, ax2 = pu.LearningCurveInSample(scores, k, step)

[[-7 12]
 [-6 11]]
[ 3.99483375  1.25868898  3.95130551  2.34171133  5.15642506  2.41898647
  2.93682886  3.33281488  2.40058572  5.08780015  3.57885463  3.98613906
  1.38125972  1.27948058  2.154576    3.45194955  5.21029667  1.6323549
  2.25024344  2.62013917  1.26118887  2.63693315  3.67398258  1.81072341
  4.7497051   1.33748061  4.25411726  5.17017349  3.5398253   2.89302399
  2.58439002  1.43156912  4.0816825   1.02177818  1.26118887  2.3918641
  2.52610816  1.44913767  2.26994916  2.65528377  4.67172192  4.83823785
  2.75118443  4.53465254  0.96018459  1.27828536  4.8942736   4.85494927
  2.63992766  4.78297441  2.63734209  4.50492333  1.55387879  1.41509662
  1.352886    1.37935846  3.96284488  2.03973744  5.08472222  4.90794904
  5.14425611  1.48016561  3.82885754  3.77903511  4.76800749  5.24700613
  2.82344212  5.221765    1.28892388  1.26614452  2.48364011  1.40076468
  5.09603744  2.95375653  5.11196284  4.37540738  3.43905329  1.91040768
  4.1051191   2.50776376  4.118425

KeyboardInterrupt: 

# LEARNING CURVE IN SAMPLE (ENRICHED)

In [2]:
import sys
sys.path.append("../../src")
import features.feature_utils as fu
import models.model_utils as mu
import plots.plot_utils as pu
import pandas as pd
import math
from sklearn.model_selection import GridSearchCV
import logging
import datetime as dt
import numpy as np

model = ['forestClass',None,None]
modelOpt = {'n_estimators':20}
k= 5
pw = True
step = 1
seqRegions = [[-7,12],[-6,11]]
data = '../../data/interim/pw_hammer_prom_lib.csv'
dataEnrich= '../../data/interim/pw_mutalik_prom_lib.csv'

dfDataset , featureBox = fu.CreateFeaturesFromData(data, seqRegions, pw, shuffle=True)
dfEnrich, enrichBox = fu.CreateFeaturesFromData(dataEnrich, seqRegions, pw, shuffle=True)

X_enrich = enrichBox.values
y_enrich = dfEnrich['rank']

X = featureBox.values
y = dfDataset['rank']

parModel = mu.SetupModel(model, modelOpt)

scores = mu.LearningCurveInSampleEnriched(dfDataset, featureBox, enrichBox, y, y_enrich, parModel, k, pw, step)

fig, ax1, ax2 = pu.LearningCurveInSampleEnriched(scores, k, step)

[[-7 12]
 [-6 11]]
[[-7 12]
 [-6 11]]
[[-7 12]
 [-6 11]]
[[-7 12]
 [-6 11]]
Progress 13.6%

KeyboardInterrupt: 

# LEARNING CURVE OUT OF SAMPLE

In [3]:
import sys
sys.path.append("../../src")
import features.feature_utils as fu
import models.model_utils as mu
import plots.plot_utils as pu
import pandas as pd
import math
from sklearn.model_selection import GridSearchCV
import loggingOut
import datetime as dt
import numpy as np



model = ['SVC','poly',3]
modelOpt = { "C":1, "gamma":0.1 , 'coef0':1}
pw = True
step = 10
seqRegions = [[0,12],[-6,11]]
data= '../../data/interim/pw_mutalik_prom_lib.csv'
dataOut = ['../../data/interim/pw_anderson_prom_lib.csv','../../data/interim/pw_brewster_prom_lib.csv',
          '../../data/interim/pw_hammer_prom_lib.csv','../../data/interim/pw_inbio_prom_lib.csv']
dataOutLabels = ['anderson','brewster','hammer','inbio']
#data= '../../data/external/mut_rand_mod_lib.csv'
#dataOut = ['../../data/external/anderson_lib.csv','../../data/external/brewster_lib.csv',
#          '../../data/external/hammer_lib.csv','../../data/external/inbio_lib.csv']
#dataOutLabels = ['anderson','brewster','hammer','inbio']

parModel = mu.SetupModel(model,modelOpt)

dfDatasetShuffled , featureBox = fu.CreateFeaturesFromData(data, seqRegions, pw, shuffle=True)

dataList = []
for d in dataOut:
    dfShuffled, Box = fu.CreateFeaturesFromData(d, seqRegions, pw)
    y_out = dfShuffled['rank']
    dataList.append((dfShuffled, Box, y_out))

X = featureBox.values
y = dfDatasetShuffled['rank']


scores = mu.LearningCurveOutOfSample(dfDatasetShuffled, featureBox, y , dataList, parModel, pw=pw, step=step)

fig, ax = pu.LearningCurveOutOfSample(scores, k, dataOutLabels)

# LEARNING CURVE OUT OF SAMPLE ENRICHED

In [61]:
import sys
sys.path.append("../../src")
import features.feature_utils as fu
import models.model_utils as mu
import plots.plot_utils as pu
import pandas as pd
import math
from sklearn.model_selection import GridSearchCV
import logging
import datetime as dt
import numpy as np


model = ['forestClass',None,None]
modelOpt = {'n_estimators':10}
pw = True
step = 10
seqRegions = [[0,12],[-6,11]]
data = '../../data/interim/pw_hammer_prom_lib.csv'
dataEnrich = ['../../data/interim/pw_mutalik_prom_lib.csv']
dataOut = ['../../data/interim/pw_anderson_prom_lib.csv','../../data/interim/pw_brewster_prom_lib.csv'
          ,'../../data/interim/pw_inbio_prom_lib.csv']
dataOutLabels = ['anderson','brewster','inbio']


parModel = mu.SetupModel(model,modelOpt)
dfDataset , featureBox = fu.CreateFeaturesFromData(data, seqRegions, pw, shuffle=True)

X = featureBox.values
y = dfDataset['rank']

enrichBoxList = []
y_enrich = []

for e in dataEnrich:
    dfEnrich, enrichBox = fu.CreateFeaturesFromData(e, seqRegions, pw)
    y_en = dfEnrich['rank']
    enrichBoxList.append(enrichBox)
    y_enrich.append(y_en)
    
enrichBox = np.vstack((enrichBoxList[:]))


dataOutList = []

for d in dataOut:
    dfOut, outBox = fu.CreateFeaturesFromData(d, seqRegions, pw)
    y_out = dfOut['rank']
    dataOutList.append((dfOut, outBox, y_out))


scores = mu.LearningCurveOutOfSampleEnriched(dfDataset, featureBox, enrichBox, y, y_enrich, dataOutList, parModel, pw, step)

fig, ax = pu.LearningCurveOutOfSampleEnriched(scores, step, dataOutLabels)

Progress 55.6%rogress 27.8%

In [63]:
ax1 = fig.get_axes()

In [65]:
ax1.legend()

AttributeError: 'list' object has no attribute 'legend'

In [44]:
lgd = ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))

AttributeError: 'list' object has no attribute 'legend'

In [72]:
fig.savefig('test1.png', bbox_extra_artists=(lgd,),bbox_inches='tight')

In [10]:
figure.autolayout : True

SyntaxError: invalid syntax (<ipython-input-10-f260c45ab60a>, line 1)

In [39]:
 = fig.legend

In [71]:
lgd = fig.gca().legend(loc='center left', bbox_to_anchor=(1, 0.5))


In [75]:
ax = fig.gca()

In [86]:
ax.legend_

In [80]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()

In [84]:
ax.get_xticks

In [89]:
ax.get_xticks()

array([ 0. ,  0.2,  0.4,  0.6,  0.8,  1. ])